In [1]:
#from fin_statement_model.importers.excel_importer import ExcelImporter
import pandas as pd

import os
import sys
import json
import openai

In [2]:
def build_prompt(spreadsheet_data):
    # Convert any non-serializable objects (like datetime) to strings using default=str.
    json_str = json.dumps(spreadsheet_data, indent=2, default=str)
    
    prompt = (
        "I have a spreadsheet represented as JSON. Each object in the array represents a cell "
        "with its location, formula, precedents, dependencies, and value. For example, the JSON looks like this:\n\n"
        f"{json_str}\n\n"
        "Based on the contents, please provide your best guess of what the row names and "
        "column header names might be. Please return your answer as two additional keys to the input JSON: "
        "'row_name' and 'column_name'. "
        "Only output valid JSON."
    )
    return prompt

In [3]:
import openpyxl
import re
from openpyxl.utils.cell import coordinate_from_string

def extract_dependencies(formula):
    """
    Extract cell references (dependencies) from a formula string.
    This is a very basic regex, and may need enhancement for complex formulas.
    """
    return re.findall(r"([A-Z]+[0-9]+)", formula)

def load_excel_with_metadata(file_path):
    # Load the workbook twice:
    # one that retains the formulas (data_only=False) and one that contains calculated values (data_only=True)
    wb_formula = openpyxl.load_workbook(file_path, data_only=False)
    wb_value = openpyxl.load_workbook(file_path, data_only=True)
    
    # We're using the active sheet in this example
    sheet_formula = wb_formula.active
    sheet_value = wb_value.active

    cells_data = []

    for row in sheet_formula.iter_rows():
        for cell in row:
            # Extract column letter and row number using coordinate_from_string
            coord = coordinate_from_string(cell.coordinate)
            col_letter, row_number = coord[0], coord[1]
            
            # Get the corresponding cell from the computed values workbook.
            cell_value_obj = sheet_value[cell.coordinate]
            
            cell_data = {
                'cell_location': cell.coordinate,
                'column': col_letter,
                'row': row_number,
                'value': cell_value_obj.value,  # Calculated value from the data_only workbook.
                'formula': None,                # Full formula string (if present)
                'precedents': None,             # Logic for precedents can be implemented as needed.
                'dependencies': None,
                'column_name': None,
                'row_name': None
            }

            # If the cell contains a formula, update formula field and extract dependencies.
            if isinstance(cell.value, str) and cell.value.startswith('='):
                cell_data['formula'] = cell.value  # The full formula
                deps = extract_dependencies(cell.value)
                cell_data['dependencies'] = deps

            
            
            # Only add cells that have a value.
            if cell_data['value'] is not None:
                cells_data.append(cell_data)

    # Build a mapping from cell coordinates to their corresponding cell data dictionaries.
    cell_map = {cell['cell_location']: cell for cell in cells_data}
    # For each cell with dependencies, iterate over its dependencies and add this cell as a precedent in the referenced cell's data.
    for cell in cells_data:
        if cell.get('dependencies'):
            for dep in cell['dependencies']:
                if dep in cell_map:
                    if cell_map[dep]['precedents'] is None:
                        cell_map[dep]['precedents'] = []
                    cell_map[dep]['precedents'].append(cell['cell_location'])

    

    
    return cells_data

In [4]:
def query_openai(prompt):
    # Make sure your API key is set in the environment variable OPENAI_API_KEY
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        sys.exit("Error: The OPENAI_API_KEY environment variable is not set.")

    # Create a client instance
    client = openai.OpenAI(api_key=api_key)
    
    # Make the API call using the new interface
    response = client.chat.completions.create(
        model="o3-mini",
        #model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an assistant that helps deduce spreadsheet headers."},
            {"role": "user", "content": prompt}
        ],
    )
    
    # Extract the content from the response
    return response.choices[0].message.content

In [5]:
file_name = "101-01-90-Minute-Case-Study-Otis-Complete.xlsx"

In [6]:
cells_info = load_excel_with_metadata(file_name)

In [7]:
prompt = build_prompt(cells_info)

In [8]:
answer = query_openai(prompt)

In [9]:
data = json.loads(answer)

In [10]:
# Initialize empty dictionaries to store the mapped data
row_names = {}
column_names = {}

# Map row and column names from data using cells_info locations
for i, cell_info in enumerate(cells_info):
    row = str(cell_info['row'])  # Convert row number to string since data keys are strings
    col = cell_info['column']
    
    # Look up row and column names from data
    if row in data['row_name']:
        cells_info[i]['row_name'] = data['row_name'][row]
    if col in data['column_name']:
        cells_info[i]['column_name'] = data['column_name'][col]


In [11]:
t = pd.DataFrame(cells_info)

In [12]:
t

,cell_location,column,row,value,formula,precedents,dependencies,column_name,row_name
0,B2,B,2,Drivers:,None,None,None,None,None
1,C2,C,2,2019-12-31 00:00:00,None,[D2],None,2019-12-31,None
2,D2,D,2,2020-12-31 00:00:00,"=EOMONTH(C2,12)",[E2],[C2],2020-12-31,None
3,E2,E,2,2021-12-31 00:00:00,"=EOMONTH(D2,12)",[F2],[D2],2021-12-31,None
4,F2,F,2,2022-12-31 00:00:00,"=EOMONTH(E2,12)",[G2],[E2],2022-12-31,None
...,...,...,...,...,...,...,...,...,...
742,F126,F,126,-475.0,=F109+F114+F122+F124,[F77],"[F109, F114, F122, F124]",2022-12-31,Net Change in Cash:
743,G126,G,126,-0.0,=G109+G114+G122+G124,[G77],"[G109, G114, G122, G124]",2023-12-31,Net Change in Cash:
744,H126,H,126,0.0,=H109+H114+H122+H124,[H77],"[H109, H114, H122, H124]",2024-12-31,Net Change in Cash:
745,I126,I,126,-0.0,=I109+I114+I122+I124,[I77],"[I109, I114, I122, I124]",2025-12-31,Net Change in Cash:


In [13]:
cells_info

[{'cell_location': 'B2',
  'column': 'B',
  'row': 2,
  'value': 'Drivers:',
  'formula': None,
  'precedents': None,
  'dependencies': None,
  'column_name': None,
  'row_name': None},
 {'cell_location': 'C2',
  'column': 'C',
  'row': 2,
  'value': datetime.datetime(2019, 12, 31, 0, 0),
  'formula': None,
  'precedents': ['D2'],
  'dependencies': None,
  'column_name': '2019-12-31',
  'row_name': None},
 {'cell_location': 'D2',
  'column': 'D',
  'row': 2,
  'value': datetime.datetime(2020, 12, 31, 0, 0),
  'formula': '=EOMONTH(C2,12)',
  'precedents': ['E2'],
  'dependencies': ['C2'],
  'column_name': '2020-12-31',
  'row_name': None},
 {'cell_location': 'E2',
  'column': 'E',
  'row': 2,
  'value': datetime.datetime(2021, 12, 31, 0, 0),
  'formula': '=EOMONTH(D2,12)',
  'precedents': ['F2'],
  'dependencies': ['D2'],
  'column_name': '2021-12-31',
  'row_name': None},
 {'cell_location': 'F2',
  'column': 'F',
  'row': 2,
  'value': datetime.datetime(2022, 12, 31, 0, 0),
  'formula'

In [14]:
# Build a lookup dictionary mapping each cell location to its associated row_name (if defined)
ref_to_row_name = {}
for cell in cells_info:
    # Only add mapping if a row_name exists for the cell
    if cell.get('row_name') is not None:
        ref_to_row_name[cell['cell_location']] = cell['row_name']

# Process each cell that contains a formula starting with '='
for cell in cells_info:
    formula = cell.get('formula')
    if formula and formula.startswith('='):
        updated_formula = formula
        # Collect all referenced cell locations from both "precedents" and "dependencies"
        refs = set()
        if cell.get('precedents'):
            refs.update(cell['precedents'])
        if cell.get('dependencies'):
            refs.update(cell['dependencies'])
        
        # Replace each cell reference with the actual row_name if available
        for ref in refs:
            if ref in ref_to_row_name:
                # Replace all occurrences of the cell reference (as a substring)
                updated_formula = updated_formula.replace(ref, str(ref_to_row_name[ref]))
        
        # Save the new formula into the cell's dictionary under a new key
        cell['formula_with_row_names'] = updated_formula

for cell in cells_info:
    # Generate precedents_names by converting each precedent cell reference into its row_name if available
    refs = cell.get('precedents')
    if refs:
        precedents_names = []
        for ref in refs:
            # Use the row name if available, otherwise keep the original reference
            precedents_names.append(ref_to_row_name.get(ref, ref))
        cell['precedents_names'] = precedents_names
    else:
        cell['precedents_names'] = []

    # Generate dependent_names by converting each dependency cell reference into its row_name if available
    deps = cell.get('dependencies')
    if deps:
        dependent_names = []
        for ref in deps:
            # Use the row name if available, otherwise keep the original reference
            dependent_names.append(ref_to_row_name.get(ref, ref))
        cell['dependent_names'] = dependent_names
    else:
        cell['dependent_names'] = []


In [15]:
cells_info

[{'cell_location': 'B2',
  'column': 'B',
  'row': 2,
  'value': 'Drivers:',
  'formula': None,
  'precedents': None,
  'dependencies': None,
  'column_name': None,
  'row_name': None,
  'precedents_names': [],
  'dependent_names': []},
 {'cell_location': 'C2',
  'column': 'C',
  'row': 2,
  'value': datetime.datetime(2019, 12, 31, 0, 0),
  'formula': None,
  'precedents': ['D2'],
  'dependencies': None,
  'column_name': '2019-12-31',
  'row_name': None,
  'precedents_names': ['D2'],
  'dependent_names': []},
 {'cell_location': 'D2',
  'column': 'D',
  'row': 2,
  'value': datetime.datetime(2020, 12, 31, 0, 0),
  'formula': '=EOMONTH(C2,12)',
  'precedents': ['E2'],
  'dependencies': ['C2'],
  'column_name': '2020-12-31',
  'row_name': None,
  'formula_with_row_names': '=EOMONTH(C2,12)',
  'precedents_names': ['E2'],
  'dependent_names': ['C2']},
 {'cell_location': 'E2',
  'column': 'E',
  'row': 2,
  'value': datetime.datetime(2021, 12, 31, 0, 0),
  'formula': '=EOMONTH(D2,12)',
  'pr

In [37]:
from fin_statement_model.financial_statement import FinancialStatementGraph

def create_financial_statement(cells_info):
    """
    Create a FinancialStatementGraph object from a list of cell dictionaries.
    Each cell dict is expected to have keys such as 'row_name', 'column_name', and 'value'.

    This function groups cells by the 'row_name' (representing the financial statement item) and
    constructs a mapping of periods (from 'column_name') to values.
    
    Args:
        cells_info (list): List of dictionaries representing Excel cells.
            Example entry:
            {
                'cell_location': 'H63',
                'column': 'H',
                'row': 63,
                'value': 22,
                'formula': '=G63',
                'precedents': ['I63', 'H66'],
                'dependencies': ['G63'],
                'column_name': '2024',
                'row_name': 'Other Income / (Expense):',
                'formula_with_row_names': '=Other Income / (Expense):',
                'precedents_names': ['Other Income / (Expense):', 'Pre-Tax Income:'],
                'dependent_names': ['Other Income / (Expense):']
            },
            ...

    Returns:
        FinancialStatementGraph: The populated financial statement graph.
    """
    # Group cells by row_name to aggregate values per financial statement item
    def clean_text(text: str, chars_to_remove=None) -> str:
        # Ensure the text is not None to avoid AttributeError
        if text is None:
            return ""
        if chars_to_remove is None:
            chars_to_remove = [":"]
        for ch in chars_to_remove:
            text = text.replace(ch, "")
        return text.strip()
    
    from datetime import datetime, date
    # First, group cells by the cleaned row_name while tracking the minimum row index and cell values.
    grouped_items = {}
    for cell in cells_info:
        # Clean the item name using the generic clean_text method to remove standard characters like ':'
        item_name = clean_text(cell.get('row_name'))
        period = cell.get('column_name', '')
        value = cell.get('value')
        row_num = cell.get('row', float('inf'))
    
        if not item_name or not period:
            continue
    
        if item_name not in grouped_items:
            grouped_items[item_name] = {"min_row": row_num, "values": {}}
        else:
            grouped_items[item_name]["min_row"] = min(grouped_items[item_name]["min_row"], row_num)
        grouped_items[item_name]["values"][period] = value

    # Build the final items dictionary,
    # skipping rows that are entirely dates if they are not the first date-only row.
    items = {}
    unique_periods = set()
    for name, data in grouped_items.items():
        if all(isinstance(v, (datetime, date)) for v in data["values"].values()):
            continue
        items[name] = data["values"]
        unique_periods.update(data["values"].keys())

    # Sort periods (assuming they are sortable, e.g., year strings) to pass to FinancialStatementGraph
    sorted_periods = sorted(unique_periods)

    # Create the FinancialStatementGraph with the detected periods
    fsg = FinancialStatementGraph(periods=sorted_periods)

    # Add each financial statement item with its period-value mapping
    for item, values in items.items():
        fsg.add_financial_statement_item(item, values)

    return fsg

In [38]:
f = create_financial_statement(cells_info)

In [40]:
f.to_dataframe()

,2019-12-31,2020-12-31,2021-12-31,2022-12-31,2023-12-31,2024-12-31,2025-12-31,2026-12-31
Products,5648.0,5371.0,6428.0,6749.400000,7.165613e+03,7.534130e+03,7.878080e+03,8.114422e+03
Services,7470.0,7385.0,7870.0,8430.344000,8.987168e+03,9.580771e+03,1.016424e+04,1.078324e+04
Total Revenue,13118.0,12756.0,14298.0,15179.744000,1.615278e+04,1.711490e+04,1.804232e+04,1.889766e+04
Cost of Products,-4640.0,-4439.0,-5293.0,-5560.232238,-5.903113e+03,-6.206702e+03,-6.490051e+03,-6.684753e+03
Cost of Services,-4652.0,-4538.0,-4812.0,-5195.009409,-5.538140e+03,-5.903934e+03,-6.263483e+03,-6.644930e+03
Operating Expenses,-1973.0,-2076.0,-2107.0,-2330.162733,-2.479529e+03,-2.627219e+03,-2.769582e+03,-2.900881e+03
Operating Income,1853.0,1703.0,2086.0,2094.339620,2.232000e+03,2.377047e+03,2.519203e+03,2.667100e+03
Other Income / (Expense),-39.0,-64.0,22.0,22.000000,2.200000e+01,2.200000e+01,2.200000e+01,2.200000e+01
Non-Service Pension Expense,33.0,-6.0,-11.0,-11.000000,-1.100000e+01,-1.100000e+01,-1.100000e+01,-1.100000e+01
Interest Income / (Expense),14.0,-122.0,-136.0,-181.825000,-2.107442e+02,-2.061446e+02,-2.350115e+02,-2.293127e+02


In [41]:
print(f)

Financial Statement Graph Summary

Periods: 2019-12-31, 2020-12-31, 2021-12-31, 2022-12-31, 2023-12-31, 2024-12-31, 2025-12-31, 2026-12-31

Financial Statement Items:
-------------------------
• Products
• Services
• Total Revenue
• Cost of Products
• Cost of Services
• Operating Expenses
• Operating Income
• Other Income / (Expense)
• Non-Service Pension Expense
• Interest Income / (Expense)
• Pre‐Tax Income
• Income Taxes
• Net Income
• (-) NCI Net Income
• Net Income to Parent
• Cash
• Accounts Receivable
• Inventory & Other
• Net PP&E, Goodwill & Intangibles
• Op. Lease Assets
• Other Assets
• Total Assets
• Accounts Payable
• Accrued Liabilities
• Contract Liabilities
• Total Debt
• Op. Lease Liabilities
• Other Liabilities
• Total Liabilities
• Common Shareholders' Equity
• Noncontrolling Interests
• Total Equity
• TOTAL LIABILITIES + EQUITY
• Balance Check
• (+/-) Reversal of NCI Net Income
• (+) D&A
• (+/-) Pensions/Other Items
• (+/-) Change in WC & Leases
• Cash Flow from Ope